In [91]:
# import the library
import pandas as pd
import numpy as np
import os
import glob
from fuzzywuzzy import process

In [92]:
bike_dir = 'Z:/AT Data/Data_to_be_processed/bicycle/'

# Initialize a list to store the filtered and combined DataFrames
combined_data = []

# Traverse the directories recursively
for intersection_name in os.listdir(bike_dir):
    intersection_path = os.path.join(bike_dir, intersection_name)
    if os.path.isdir(intersection_path):
        for folder_level_2 in os.listdir(intersection_path):
            level_2_path = os.path.join(intersection_path, folder_level_2)
            if os.path.isdir(level_2_path):
                for folder_level_3 in os.listdir(level_2_path):
                    level_3_path = os.path.join(level_2_path, folder_level_3)
                    if os.path.isdir(level_3_path):
                        for file in os.listdir(level_3_path):
                            if file.endswith('.csv'):
                                file_path = os.path.join(level_3_path, file)

                                # Determine the direction from the CSV filename
                                if "Southbound" in file:
                                    direction = "Southbound"
                                elif "South" in file:
                                    direction = "Southbound"
                                elif "Northbound" in file:
                                    direction = "Northbound"
                                elif "North" in file:
                                    direction = "Northbound"
                                elif "Westbound" in file:
                                    direction = "Westbound"
                                elif "West" in file:
                                    direction = "Westbound"
                                elif "Eastbound" in file:
                                    direction = "Eastbound"
                                elif "East" in file:
                                    direction = "Eastbound"
                                else:
                                    direction = "Unknown"

                                # Read the CSV file
                                df = pd.read_csv(file_path)
                                # Step 2: Add required "Direction" and "Intersection" columns
                                df["Direction"] = direction
                                df["Intersection"] = intersection_name

                                # Append the cleaned DataFrame to the list
                                combined_data.append(df)

# Step 3: Combine all the filtered DataFrames into one and ignore column mismatches
final_combined_df = pd.concat(combined_data, ignore_index=True)
final_combined_df.head()

,"=T(""Date"")","=T(""Bike Lane"")",Direction,Intersection,"=T(""Westbound"")","=T(""Eastbound"")"
0,"=T(""05/15/2024 00:00:00"")",NaN,Northbound,880 NB Ramp & 23rd Ave,NaN,NaN
1,"=T(""05/15/2024 01:00:00"")",NaN,Northbound,880 NB Ramp & 23rd Ave,NaN,NaN
2,"=T(""05/15/2024 02:00:00"")",NaN,Northbound,880 NB Ramp & 23rd Ave,NaN,NaN
3,"=T(""05/15/2024 03:00:00"")",NaN,Northbound,880 NB Ramp & 23rd Ave,NaN,NaN
4,"=T(""05/15/2024 04:00:00"")",NaN,Northbound,880 NB Ramp & 23rd Ave,NaN,NaN


In [93]:
# remove the special character from the column heading
final_combined_df.columns = final_combined_df.columns.str.replace(r'=T\("([^"]+)"\)', r'\1', regex=True)
# drop the empty rows
final_combined_df1=final_combined_df.dropna(subset=['Bike Lane', 'Westbound','Eastbound'], how='all')
# fill the bike lane with west nd east bound sum
final_combined_df1['Bike Lane']=final_combined_df1['Bike Lane'].fillna(final_combined_df1['Eastbound'] +final_combined_df1['Westbound'])
# now drop the easta nd west bound
final_combined_df1=final_combined_df1.drop(['Westbound','Eastbound'],axis=1)
# fix the date column as datetime format
final_combined_df1['Date'] = final_combined_df1['Date'].str.extract(r'(\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2})')[0]
final_combined_df1['Date'] = pd.to_datetime(final_combined_df1['Date'])
final_combined_df1.head()

C:\Users\s160555\AppData\Local\Temp\ipykernel_23020\4261085371.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_combined_df1['Bike Lane']=final_combined_df1['Bike Lane'].fillna(final_combined_df1['Eastbound'] +final_combined_df1['Westbound'])


,Date,Bike Lane,Direction,Intersection
34,2024-05-16 10:00:00,0.0,Northbound,880 NB Ramp & 23rd Ave
35,2024-05-16 11:00:00,3.0,Northbound,880 NB Ramp & 23rd Ave
36,2024-05-16 12:00:00,0.0,Northbound,880 NB Ramp & 23rd Ave
37,2024-05-16 13:00:00,1.0,Northbound,880 NB Ramp & 23rd Ave
38,2024-05-16 14:00:00,1.0,Northbound,880 NB Ramp & 23rd Ave


In [94]:
final_combined_df1.Intersection.unique()

array(['880 NB Ramp & 23rd Ave', 'High St & Oakport St',
       'HWY 49 & Edgewood Rd', 'HWY 50 & Bedford Ave',
       'HWY 50 & Canal St', 'HWY 50 & Spring St'], dtype=object)

In [95]:
final_combined_df1.shape

(48198, 4)

In [96]:
# read the meta data
meta_data_path4=pd.read_excel(r'Z:/AT Data/Data_to_be_processed/DATA LOCATION GUIDE.xlsx',sheet_name='D4')
meta_data4=meta_data_path4[['ID','Intersection Description']]
meta_data_path3=pd.read_excel(r'Z:/AT Data/Data_to_be_processed/DATA LOCATION GUIDE.xlsx',sheet_name='D3')
meta_data3=meta_data_path3[['ID','Intersection Description']]
meta_data34=pd.concat([meta_data3,meta_data4])
meta_data34['District'] = meta_data34['ID'].astype(str).str[0].astype(int)
meta_data34.head()

,ID,Intersection Description,District
0,3001,HWY 49 & I-80 EB off Ramp,3
1,3002,HWY 49 & Sawyer St,3
2,3003,HWY 49 & Elm Ave/Fulweiler Ave,3
3,3004,HWY 49 & Palm Ave,3
4,3005,HWY 49 & Marguerite Mile Rd,3


In [97]:
meta_data34['Intersection Description'].unique()

array(['HWY 49 & I-80 EB off Ramp', 'HWY 49 & Sawyer St',
       'HWY 49 & Elm Ave/Fulweiler Ave', 'HWY 49 & Palm Ave',
       'HWY 49 & Marguerite Mile Rd', 'HWY 49 & Edgewood Rd',
       'HWY 49 & Luther Rd', 'HWY 49 & New Airport Rd',
       'HWY 49 & Atwood Rd', 'HWY 49 & Willow Creek Dr',
       'HWY 49 & Bell Rd', 'HWY 49 & Education St', 'HWY 49 & Quartz Dr',
       'HWY 49 & Locksley Ln', 'HWY 49 & Shale Ridge Rd',
       'HWY 49 & Dry Creek Rd', 'HWY 50 & Spring St', 'HWY 50 & Canal St',
       'HWY 50 & Bedford Ave', 'Central & EB580',
       'CentralWB & San Joaquin-Jacuzzi', 'El Camino Real & Churchill',
       'El Camino Real & Embarcadero', 'El Camino Real & Medical Center',
       'El Camino Real & Serra Park', 'Frontage & Access',
       'I-80 E Ramp & Buchanan St', 'I-80 W Ramp & Buchanan St',
       'Mountain Blvd & R13', 'Potrero & I-80 EB', 'Powell & I-80 EB',
       'Powell & I-80 WB', 'RTE 123- San Pablo Ave & Marin Ave',
       'W Grand & Frontage', '880 NB Ramp 

In [99]:
final_combined_df2=pd.merge(final_combined_df1,meta_data34, left_on=['Intersection'], right_on=['Intersection Description'], how='inner')
final_combined_df2.head()

,Date,Bike Lane,Direction,Intersection,ID,Intersection Description,District
0,2024-05-16 10:00:00,0.0,Northbound,880 NB Ramp & 23rd Ave,4016,880 NB Ramp & 23rd Ave,4
1,2024-05-16 11:00:00,3.0,Northbound,880 NB Ramp & 23rd Ave,4016,880 NB Ramp & 23rd Ave,4
2,2024-05-16 12:00:00,0.0,Northbound,880 NB Ramp & 23rd Ave,4016,880 NB Ramp & 23rd Ave,4
3,2024-05-16 13:00:00,1.0,Northbound,880 NB Ramp & 23rd Ave,4016,880 NB Ramp & 23rd Ave,4
4,2024-05-16 14:00:00,1.0,Northbound,880 NB Ramp & 23rd Ave,4016,880 NB Ramp & 23rd Ave,4


In [26]:
# now join with intersection details 
# if 'Intersection Description' in meta_data34.columns:
#     intersection_names=meta_data34['Intersection Description'].tolist()

#     def match_intersection(name):
#         match, score = process.extractOne(name, intersection_names)  # Find best match
#         return match if score > 80 else None 
#             # Apply matching function to the final_df
#     final_combined_df1['Matched_Intersection'] = final_combined_df1['Intersection'].apply(match_intersection)

#     # Merge with intersection description data
#     final_combined_df1 = final_combined_df1.merge(meta_data34, left_on='Matched_Intersection', right_on='Intersection Description', how='left')

# else:
#     print("Error: 'Intersection Description' column not found in the intersection description file.")
# final_combined_df1.head()

,Date,Bike Lane,Direction,Intersection,Matched_Intersection,ID,Intersection Description,District
0,2024-05-16 10:00:00,0.0,Northbound,880 NB Ramp & 23rd Ave,880 NB Ramp &23rd Ave,4016,880 NB Ramp &23rd Ave,4
1,2024-05-16 11:00:00,3.0,Northbound,880 NB Ramp & 23rd Ave,880 NB Ramp &23rd Ave,4016,880 NB Ramp &23rd Ave,4
2,2024-05-16 12:00:00,0.0,Northbound,880 NB Ramp & 23rd Ave,880 NB Ramp &23rd Ave,4016,880 NB Ramp &23rd Ave,4
3,2024-05-16 13:00:00,1.0,Northbound,880 NB Ramp & 23rd Ave,880 NB Ramp &23rd Ave,4016,880 NB Ramp &23rd Ave,4
4,2024-05-16 14:00:00,1.0,Northbound,880 NB Ramp & 23rd Ave,880 NB Ramp &23rd Ave,4016,880 NB Ramp &23rd Ave,4


In [27]:
final_combined_df1.columns

Index(['Date', 'Bike Lane', 'Direction', 'Intersection',
       'Matched_Intersection', 'ID', 'Intersection Description', 'District'],
      dtype='object')

In [28]:
final_combined_df1.shape

(48198, 8)

In [100]:
# we will drop unnecessary columns
com_df1=final_combined_df2[['Date', 'Bike Lane', 'Direction', 'ID', 'Intersection Description', 'District']]
com_df2=com_df1.rename(columns={'ID':'Loc_id','Intersection Description':'Intersection'})
com_df2=com_df2.rename(columns={'Bike Lane':'Count'})
com_df2.head()

,Date,Count,Direction,Loc_id,Intersection,District
0,2024-05-16 10:00:00,0.0,Northbound,4016,880 NB Ramp & 23rd Ave,4
1,2024-05-16 11:00:00,3.0,Northbound,4016,880 NB Ramp & 23rd Ave,4
2,2024-05-16 12:00:00,0.0,Northbound,4016,880 NB Ramp & 23rd Ave,4
3,2024-05-16 13:00:00,1.0,Northbound,4016,880 NB Ramp & 23rd Ave,4
4,2024-05-16 14:00:00,1.0,Northbound,4016,880 NB Ramp & 23rd Ave,4


In [101]:
# make sure the date is in date format to extract the temporal variable
com_df2['Date'] = pd.to_datetime(com_df2['Date'], format="%m/%d/%Y %H:%M:%S")

# Extract year, month, day
com_df2['Year'] = com_df2['Date'].dt.year
com_df2['Month'] = com_df2['Date'].dt.strftime('%b') 
com_df2['Day'] = com_df2['Date'].dt.day_name()

# Extract weekday name and check if it's a weekend
com_df2['Weekday_Type'] = com_df2['Day'].apply(lambda x: "Weekend" if x in ['Saturday', 'Sunday'] else "Weekday")

# Extract hour and AM/PM
com_df2['Hour'] = com_df2['Date'].dt.strftime('%I %p').str.lstrip('0')
com_df2['Mode']="Bicycle"
com_df3=com_df2[['Loc_id','District','Intersection','Year', 'Month',
       'Day','Weekday_Type','Date', 'Hour','Mode','Direction','Count']]
com_df3.head()

,Loc_id,District,Intersection,Year,Month,Day,Weekday_Type,Date,Hour,Mode,Direction,Count
0,4016,4,880 NB Ramp & 23rd Ave,2024,May,Thursday,Weekday,2024-05-16 10:00:00,10 AM,Bicycle,Northbound,0.0
1,4016,4,880 NB Ramp & 23rd Ave,2024,May,Thursday,Weekday,2024-05-16 11:00:00,11 AM,Bicycle,Northbound,3.0
2,4016,4,880 NB Ramp & 23rd Ave,2024,May,Thursday,Weekday,2024-05-16 12:00:00,12 PM,Bicycle,Northbound,0.0
3,4016,4,880 NB Ramp & 23rd Ave,2024,May,Thursday,Weekday,2024-05-16 13:00:00,1 PM,Bicycle,Northbound,1.0
4,4016,4,880 NB Ramp & 23rd Ave,2024,May,Thursday,Weekday,2024-05-16 14:00:00,2 PM,Bicycle,Northbound,1.0


In [102]:
com_df3['Direction'].unique()

array(['Northbound', 'Southbound', 'Eastbound'], dtype=object)

In [103]:
# Check the duplicate values and drop it
com_df4=com_df3.drop_duplicates(subset=['Loc_id','District','Intersection','Year', 'Month',
       'Day','Weekday_Type','Date', 'Hour','Mode','Direction','Count'])
com_df4.shape

(48198, 12)

In [104]:
com_df4.isnull().sum()

Loc_id          0
District        0
Intersection    0
Year            0
Month           0
Day             0
Weekday_Type    0
Date            0
Hour            0
Mode            0
Direction       0
Count           0
dtype: int64

In [105]:
com_df4['Count'].describe()

count    48198.000000
mean         1.031516
std          2.634175
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max         69.000000
Name: Count, dtype: float64

In [106]:
fraction_values = com_df4[com_df4['Count'] != com_df4['Count'].astype(int)]
print(fraction_values)

Empty DataFrame
Columns: [Loc_id, District, Intersection, Year, Month, Day, Weekday_Type, Date, Hour, Mode, Direction, Count]
Index: []


In [107]:
com_df3.to_csv('Z:/AT Data/Processed_data/D34_Bike/D34_bike_data_for_Tableau.csv')

In [108]:
# Lets extract the data for Traffic Census
ts=com_df4[['Loc_id','Date', 'Hour','Direction','Count']]
ts['Count'] = ts['Count'].astype(int)
ts=ts.rename(columns={'Hour':'Time'})
ts['Date'] = pd.to_datetime(ts['Date']).dt.date
# Pivot the table
pivoted_df = ts.pivot_table(index=['Loc_id','Date', 'Time'], columns='Direction', values='Count', fill_value=0)
# Reset index to make it more readable
pivoted_df = pivoted_df.reset_index()
pivoted_df.head()

C:\Users\s160555\AppData\Local\Temp\ipykernel_23020\1890048495.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts['Count'] = ts['Count'].astype(int)


Direction,Loc_id,Date,Time,Eastbound,Northbound,Southbound
0,3006,2024-10-01,1 AM,0.0,0.0,0.0
1,3006,2024-10-01,1 PM,2.0,0.0,0.0
2,3006,2024-10-01,10 AM,0.0,0.0,0.0
3,3006,2024-10-01,10 PM,0.0,0.0,0.0
4,3006,2024-10-01,11 AM,2.0,0.0,0.0


In [109]:
pivoted_df.to_csv('Z:/AT Data/Processed_data/D34_Bike/D34_bike_data_for_census.csv',index=False)

In [110]:
# end of the script